# Queries with and without Azure OpenAI

Now that we have our Search Engine loaded and running, we are going to try some example queries and then use Azure OpenAI service to see if we can get even better results

## Set up variables

In [1]:
api_version = '2021-04-30-Preview'

# endpoint = os.environ.get("SEARCH_ENDPOINT")
# api_key = os.getenv("SEARCH_KEY")

endpoint = "https://azure-cog-search-pabdyosydd7ta.search.windows.net"
api_key = "DDDUwtXOCSOjm1fPBRLNFofKEEvxXDpGF0Sy4S3ktjAzSeAgbz9Q"
index_name = "cogsrch-index"

In [2]:
# Setup the Payloads header
api_version = '2021-04-30-Preview'
headers = {'Content-Type': 'application/json','api-key': api_key}
params = {'api-version': api_version}

## Without Azure OpenAI

In [19]:
QUESTION = "What is Reinforcment Lrning" # Notice that is misspelled intentionally

In [20]:
import urllib
import requests
from IPython.display import display, HTML

url = endpoint + '/indexes/'+ index_name + '/docs'
url += '?api-version={}'.format(api_version)
url += '&search={}'.format(QUESTION)
url += '&select=pages'
url += '&$top=5'
url += '&queryLanguage=en-us'
url += '&queryType=semantic'
url += '&semanticConfiguration=my-semantic-config'
url += '&$count=true'
url += '&speller=lexicon'
url += '&answers=extractive|count-3'
url += '&captions=extractive|highlight-true'
url += '&highlightPreTag=' + urllib.parse.quote('<span style="background-color: #f5e8a3">', safe='')
url += '&highlightPostTag=' + urllib.parse.quote('</span>', safe='')

resp = requests.get(url, headers=headers)
print(url)
print(resp.status_code)

search_results = resp.json()
print("Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))
print("Highest Search Score: {}".format(search_results['value'][0]['@search.score']))

https://azure-cog-search-pabdyosydd7ta.search.windows.net/indexes/cogsrch-index/docs?api-version=2021-04-30-Preview&search=Why Satya is excited?&select=pages&$top=5&queryLanguage=en-us&queryType=semantic&semanticConfiguration=my-semantic-config&$count=true&speller=lexicon&answers=extractive|count-3&captions=extractive|highlight-true&highlightPreTag=%3Cspan%20style%3D%22background-color%3A%20%23f5e8a3%22%3E&highlightPostTag=%3C%2Fspan%3E
200
Results Found: 4, Results Returned: 4
Highest Search Score: 5.1673346


In [27]:
# Answers from semantic Search
display(HTML('<h4>Top Answers (Semantic Search)</h4>'))
for result in search_results['@search.answers']:
    if result['score'] > 0.5:
        display(HTML('<h5>' + 'Answer - score: ' + str(result['score']) + '</h5>'))
        display(HTML(result['text']))

        
# Results from semantic search
file_content = dict()

print("\n\n")
display(HTML('<h4>Top Results (Key-Word Search)</h4>'))
for result in search_results['value']:
    if result['@search.rerankerScore'] > 0.3:
        display(HTML('<h5>' + result['metadata_storage_name'] + '&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;score: ' + str(result['@search.rerankerScore']) + '</h5>'))
        for caption in result['@search.captions']:
            # print(caption)
            display(HTML(caption['text']))
        file_content[result['metadata_storage_path']]=result['pages']

## Comments on Query results

As seen above the semantic search feature of Azure Cognitive Search service is pretty good. It gives us the top answers and also the top results with the corresponding file and the paragraph where the answers is possible located
Let's see if we can make this better with Azure OpenAI

## Using Azure OpenAI

Of course we want OpenAI to give a better answer chat style, so we instead of sending this results, we send the content of this articles to OpenAI and lets GPT model give the answer.

The problem is that the content of the search result files is or can be very lengthy, more than the 4096 tokens allowed by the GPT Azure OpenAI models. So what we need to do is to split in chunks, vectorize and do a vector semantic search. 
Let's do that..

In [28]:
# Import libraries
from App.app.embeddings import OpenAIEmbeddings
from langchain.chains import VectorDBQAWithSourcesChain
from langchain.llms import AzureOpenAI
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = "48e3114b81d1430eb1f3df7fb783f176"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://pablo.openai.azure.com/" 


In [29]:
embeddings = OpenAIEmbeddings()

In [30]:
docs = []
for key,value in file_content.items():
    for page in value:
        docs.append(Document(page_content=page, metadata={"source": key}))

In [31]:
db = Chroma.from_documents(docs, embeddings)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [32]:
chain = VectorDBQAWithSourcesChain.from_chain_type(AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0),
                                                   chain_type="stuff", vectorstore=db)


Exiting: Cleaning up .chroma directory


In [33]:
chain({"question": QUESTION})

NotEnoughElementsException: Number of requested results 4 cannot be greater than number of elements in index 3

##### This answer is way better than taking just the result from Azure Cognitive Search. So the summary is:
- Azure Cognitive Search give us the top results
- Azure OpenAI construct the best answer and source best on the Search Results
- Best of two worlds!